In [2]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import HBox, VBox
from IPython.display import display
%matplotlib inline

# Wavelet characterization of spatial pattern in allele frequency
## Jesse Lasky, Diana Gamba, Timothy Kiett

## The problem

Want to better understand spatial patterns of allele frequency (eg, local adaptation, range expansion), but 

1. don't want to have to define discrete 'populations' (eg, $F_{ST}$, EEMS)
    - arbitrary
    - might not be correct spatial scale

2. ordinations (eg, PCA) "heavily influenced by global outliers of genetic divergence" (??)

## The solution

Wavelets: localized, scale-specific patterns

In music, this is the volume of a note (of a given frequency) heard at a given time.

Here, this is the rate of allele frequency change (at a given scale) at a given location.

![image](images/fig-s1.png)

## The details

### Gaussian smoothing function
$$
\eta_{a,b}^s(x,y) = \frac{k\left(\frac{x-a}{s},\frac{y-b}{s}\right)}{\sum_{(u,v)\in\Omega}k\left(\frac{u-a}{s},\frac{v-b}{s}\right)}
$$
where
$$k(x,y)=e^{-(x^2+y^2)/2}$$
is a Gaussian kernel and
$$\Omega=\{(u_1,v_1),(u_2,v_2),...,(u_n,v_n)\}$$
is the set of sampling points.

In [3]:
def k(x):
    return np.exp(-(sum(x**2))/2)

def eta(x,a,s,omega):
    return k((x-a)/s)/sum(list(map(k,omega/s)))

In [4]:
@widgets.interact(a=(0.0,9),s=(0.1,10))
def plot_eta(a=4.5,s=1):
    omega = np.array(np.meshgrid(range(10))).T.reshape(-1,1)
    xs = np.linspace(0,9,100)
    etas = np.array([eta(i,a=np.array([a]),s=s,omega=omega) for i in xs])
    fig, ax = plt.subplots()
    ax.plot(xs,etas)
    ax.set_ylabel(r'$\eta_a^s(x)$')
    ax.set_xlabel(r'$x$')
    plt.show()

interactive(children=(FloatSlider(value=4.5, description='a', max=9.0), FloatSlider(value=1.0, description='s'…

### Difference of Gaussians wavelet filter
$$
\psi_{a,b}^s(x,y) = \eta_{a,b}^s(x,y) - \eta_{a,b}^{\beta s}(x,y)
$$
with $\beta>1$.

In [5]:
def psi(x,a,s,omega,beta):
    return eta(x,a,s,omega) - eta(x,a,beta*s,omega)

In [7]:
@widgets.interact(a=(0.0,9),s=(0.1,10),beta=(1.0,10))
def plot_psi(a=4.5,s=1,beta=1.87):
    omega = np.array(np.meshgrid(range(10))).T.reshape(-1,1)
    xs = np.linspace(0,9,100)
    etas = np.array([eta(i,a=np.array([a]),s=s,omega=omega) for i in xs])
    etas2 = np.array([eta(i,a=np.array([a]),s=s*beta,omega=omega) for i in xs])
    psis = np.array([psi(i,a=np.array([a]),s=s,omega=omega,beta=beta) for i in xs])
    fig, ax = plt.subplots()
    ax.plot(xs,etas, label=r'$\eta_a^s(x)$')
    ax.plot(xs,etas2, label=r'$\eta_a^{\beta s}(x)$')
    ax.plot(xs,psis, label=r'$\psi_a^s(x)$')
#     ax.set_ylabel(r'$\psi_a^s(x)$')
    ax.set_xlabel(r'$x$')
    plt.legend()
    plt.show()

interactive(children=(FloatSlider(value=4.5, description='a', max=9.0), FloatSlider(value=1.0, description='s'…

### Adaptive wavelet transform
$$
(T^{wav}f_i)(a,b,s) = \frac{1}{h_{a,b}(s)}\sum_{(u,v)\in\Omega}\psi_{a,b}^s(u,v) f_i(u,v)
$$
where
$$
h_{a,b}(s) = \sqrt{\sum_{(u,v)\in\Omega}[\psi_{a,b}^s(u,v)]^2}
$$
is a normalization constant.

In [8]:
def h(a,s,omega,beta):
    return sum([psi(u,a,s,omega,beta)**2 for u in omega])
    
def Twavfi(a,s,omega,beta,fi):
    return sum([psi(u,a,s,omega,beta)*fi(u) for u in omega])/h(a,s,omega,beta)

In [9]:
@widgets.interact(a=(0.0,9),s=(0.1,10))
def plot_Twavi(a=4.5,s=1):
    beta=1.87
    omega = np.array(np.meshgrid(range(10))).T.reshape(-1,1)
    def fi(x): return np.sin(x)/2 + 1/2
    xs = np.linspace(0,9,100)
    fig, axs = plt.subplots(1,2,figsize=(10,5))
    axs[0].plot(xs,[fi(x) for x in xs], label="$f_i(x)$")
    axs[0].plot(xs,[psi(x,a=np.array([a]),s=s,omega=omega,beta=beta) for x in xs], label=r'$\psi_a^s(x)$')
    axs[1].plot(xs,[Twavfi(a=x,s=s,omega=omega,beta=beta,fi=fi) for x in xs])
    axs[0].set_xlabel(r'$x$')
    axs[1].set_xlabel(r'$a$')
    axs[1].set_ylabel(r'$(T^{wav}f_i)(a,s)$')
    axs[0].legend()
    plt.show()

interactive(children=(FloatSlider(value=4.5, description='a', max=9.0), FloatSlider(value=1.0, description='s'…

### Wavelet genetic dissimilarity
$$
D_{a,b}^{wav}(s) = \sqrt{\sum_{i=1}^I [(T^{wav}f_i)(a,b,s)]^2}
$$

In [10]:
def Dwav(a,s,omega,beta,fis):
    return sum([Twavfi(a,s,omega,beta,fi)**2 for fi in fis])**0.5

In [11]:
@widgets.interact(a=(0.0,9),s=(0.1,10))
def plot_Twavi(a=4.5,s=1):
    beta=1.87
    omega = np.array(np.meshgrid(range(10))).T.reshape(-1,1)
    def f1(x): return np.sin(x)/2 + 1/2
    def f2(x): return np.sin(x/np.pi)/2 + 1/2
    fis = [f1,f2]
    xs = np.linspace(0,9,100)
    fig, axs = plt.subplots(1,2,figsize=(10,5))
    axs[0].plot(xs,[f1(x) for x in xs], label='$f_1(x)$')
    axs[0].plot(xs,[f2(x) for x in xs], label='$f_2(x)$')
    axs[0].plot(xs,[psi(x,a=np.array([a]),s=s,omega=omega,beta=beta) for x in xs], label=r'$\psi_a^s(x)$')
    axs[1].plot(xs,[Dwav(a=x,s=s,omega=omega,beta=beta,fis=fis) for x in xs])
    axs[0].set_xlabel(r'$x$')
    axs[1].set_xlabel(r'$a$')
    axs[1].set_ylabel(r'$(D^{wav}_a)(s)$')
    axs[0].legend()
    plt.show()

interactive(children=(FloatSlider(value=4.5, description='a', max=9.0), FloatSlider(value=1.0, description='s'…

### Scaled wavelet variance
$$
var((T^{wav}f_i)(a,b,s)/sd(f_i))
$$

In [12]:
def varT(s,omega,beta,fi):
    fs = [fi(u) for u in omega]
    Ts = [Twavfi(u,s,omega,beta,fi) for u in omega]
    return np.var(Ts/np.var(fs)**0.5)

In [20]:
@widgets.interact(a=(0.0,9),s=(0.1,10))
def plot_varT(a=4.5,s=1):
    beta=1.87
    omega = np.array(np.meshgrid(range(10))).T.reshape(-1,1)
    def f1(x): return np.sin(x)/2 + 1/2
    def f2(x): return np.sin(x/np.pi)/2 + 1/2
    fis = [f1,f2]
    xs = np.linspace(0,9,100)
    fig, axs = plt.subplots(1,2,figsize=(10,5))
    axs[0].plot(xs,[f1(x) for x in xs], label='$f_1(x)$')
    axs[0].plot(xs,[f2(x) for x in xs], label='$f_2(x)$')
    axs[0].plot(xs,[psi(x,a=np.array([a]),s=s,omega=omega,beta=beta) for x in xs], label=r'$\psi_a^s(x)$')
    ss = np.linspace(1,5,100)
    axs[1].plot(ss,[varT(s=s,omega=omega,beta=beta,fi=f1) for s in ss], label='$f_1$')
    axs[1].plot(ss,[varT(s=s,omega=omega,beta=beta,fi=f2) for s in ss], label='$f_2$')
    axs[0].set_xlabel(r'$x$')
    axs[1].set_xlabel(r'$s$')
    axs[1].set_ylabel(r'$var((T^{wav}f_i)(a,b,s)/sd(f_i))$')
    axs[0].legend()
    axs[1].legend()
    plt.show()

interactive(children=(FloatSlider(value=4.5, description='a', max=9.0), FloatSlider(value=1.0, description='s'…

## Simulations

![image](images/fig-1.png)

   ![image](images/fig-2.png)

   ![image](images/fig-3.png)

   ![image](images/fig-4.png)

   ![image](images/fig-5.png)

## Application

   ![image](images/fig-6.png)

   ![image](images/fig-7.png)

   ![image](images/fig-8.png)

## Discussion

Whattya think?